In [1]:
import re
import json
import hashlib
import time
from pathlib import Path
import fitz  # PyMuPDF

In [2]:
def slugify(s):
    """Convert text to a safe slug for IDs."""
    return re.sub(r'[^A-Za-z0-9]+', '-', s.strip()).strip('-')

In [12]:
def sha256_file(path):
    """Calculate SHA-256 hash of a file."""
    h = hashlib.sha256()
    with open(path, 'rb') as f:
        for b in iter(lambda: f.read(1 << 20), b''):
            h.update(b)
    return h.hexdigest()

In [4]:
def chunk_text(text, max_chars=2200, overlap=0):
    """Split text into chunks of max_chars length."""
    start = 0
    while start < len(text):
        end = min(start + max_chars, len(text))
        yield (start, end, text[start:end].strip())
        start = max(end - overlap, end)

In [5]:
def make_ids(country, visa_type, year, doc_slug, page_num, seq):
    """Generate doc_id and chunk_id."""
    doc_id = f"{country}-{slugify(visa_type)}-{year}-{doc_slug}"
    chunk_id = f"{doc_id}-Pg{page_num}-{seq:03d}"
    return doc_id, chunk_id

In [8]:
def pdf_to_chunks(pdf_path, meta, out_path):
    pdf_path = Path(pdf_path)
    if not pdf_path.exists():
        raise FileNotFoundError(f"PDF file not found: {pdf_path}")

   # Read PDF
    doc = fitz.open(str(pdf_path))
    doc_sha = sha256_file(pdf_path)
    meta["doc_sha256"] = doc_sha

    all_chunks = []
    seq = 1

    for page_num in range(len(doc)):
        page = doc[page_num]
        text = page.get_text("text")
        for cs, ce, chunk in chunk_text(text):
            doc_id, chunk_id = make_ids(meta["country"], meta["visa_type"], meta["year"], meta["doc_slug"], page_num+1, seq)
            chunk_meta = {
                "chunk_id": chunk_id,
                "doc_id": doc_id,
                "source": meta["source"],
                "url": meta["url"],
                "country": meta["country"],
                "visa_type": meta["visa_type"],
                "effective_date": meta["effective_date"],
                "version": meta["version"],
                "doc_sha256": meta["doc_sha256"],
                "retrieved_at": time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime()),
                "page": page_num+1,
                "pages": [page_num+1],
                "section_title": None,  # Optional: detect from headings
                "language": "en",
                "char_start": cs,
                "char_end": ce,
                "text": chunk
            }
            all_chunks.append(chunk_meta)
            seq += 1

    out_path = Path(out_path)
    out_path.parent.mkdir(parents=True, exist_ok=True)

    with open(out_path, "w", encoding="utf-8") as f:
        for ch in all_chunks:
            f.write(json.dumps(ch, ensure_ascii=False) + "\n")

    print(f"✅ Saved {len(all_chunks)} chunks to {out_path}")

In [11]:
if __name__ == "__main__":
    pdf_file = r"E:\Info_Srping\swiftvisa\data\raw\UK_Student_Visa.pdf"
    output_file = r"E:\Info_Srping\swiftvisa\data\processed\UK_StudentVisa_chunks.jsonl"

    meta_info = {
        "country": "UK",
        "visa_type": "Student and Child Student",
        "year": "2025",
        "doc_slug": "UK Student Visa Guide 2024",
        "source": "Student and Child Student",
        "url": "https://gov.uk/student-visa",
        "effective_date": "2025-07-16",
        "version": "11.0"
    }

    pdf_to_chunks(pdf_file, meta_info, output_file)

✅ Saved 144 chunks to E:\Info_Srping\swiftvisa\data\processed\UK_StudentVisa_chunks.jsonl
